In [1]:
import os
import boa
import warnings
import random
import pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

## Setup

In [2]:
filename = "./impl_comparison_data.csv"

Generate admin and fee receiver addresses (shared between old and new versions):

In [3]:
admin = boa.env.generate_address()
fee_receiver = boa.env.generate_address()
trader = boa.env.generate_address()
zero_address = boa.eval("empty(address)")
print(admin, fee_receiver, trader)

Generate coins and coin holders for trading ...

In [4]:
coin_a = boa.load("../../contracts/mocks/ERC20Mock.vy", "coin_a", "coin_a", 18)
coin_b = boa.load("../../contracts/mocks/ERC20Mock.vy", "coin_b", "coin_b", 18)

coin_a._mint_for_testing(trader, 10**50, sender=trader)
coin_b._mint_for_testing(trader, 10**50, sender=trader)
print(coin_a, coin_b)

Pool parameters:

In [5]:
A = 400000
gamma = 145000000000000
mid_fee = 26000000
out_fee = 45000000
allowed_extra_profit = 2000000000000
fee_gamma = 230000000000000
adjustment_step = 146000000000000
ma_time = 600
xcp_ma_time = 1800 * 24
initial_price = 10**18  # 1:1 at the start
admin_fee = 5 * 10**9

Instantiate contracts for factory:

In [6]:
old_swap_implementation = boa.load(
    "../../contracts/old/CurveCryptoSwap2ETH.vy", 
    boa.env.generate_address()  # use some random address as weth
)
old_math = boa.load("../../contracts/old/CurveCryptoSwap2Math.vy")
old_token = boa.load("../../contracts/old/CurveCryptoLPToken.vy")

with boa.env.prank(admin):
    old_factory = boa.load(
        "../../contracts/old/CurveCryptoFactory.vy",
        fee_receiver,
        old_swap_implementation,
        old_token,
        zero_address,
        zero_address
    )

old_swap = old_factory.deploy_pool(
    "old", "old", [coin_a.address, coin_b.address],
    A, gamma, mid_fee, out_fee, allowed_extra_profit, fee_gamma, 
    adjustment_step, admin_fee, ma_time, initial_price
)
old_swap = boa.load_partial("../../contracts/old/CurveCryptoSwap2ETH.vy").at(old_swap)
old_swap_token = boa.load_partial("../../contracts/mocks/ERC20Mock.vy").at(old_swap.token())
old_swap, old_swap_token

In [7]:
ng_swap_implementation = boa.load_partial("../../contracts/main/CurveTwocryptoOptimized.vy")
ng_math = boa.load("../../contracts/main/CurveCryptoMathOptimized2.vy")
ng_views = boa.load("../../contracts/main/CurveCryptoViews2Optimized.vy")
with boa.env.prank(admin):
    ng_factory = boa.load("../../contracts/main/CurveTwocryptoFactory.vy")
    ng_factory.initialise_ownership(fee_receiver, admin)

    ng_factory.set_pool_implementation(ng_swap_implementation.deploy_as_blueprint().address, 0)
    ng_factory.set_gauge_implementation(zero_address)
    ng_factory.set_views_implementation(ng_views.address)
    ng_factory.set_math_implementation(ng_math.address)

ng_swap = ng_factory.deploy_pool(
    "ng", "ng", [coin_a.address, coin_b.address],
    0, A, gamma, mid_fee, out_fee, fee_gamma, allowed_extra_profit, adjustment_step, int(ma_time/0.693), initial_price
)
ng_swap = ng_swap_implementation.at(ng_swap)
ng_swap

Grant approvals etc.

In [8]:
with boa.env.prank(trader):
    for coin in [coin_a, coin_b]:
        for pool in [old_swap, ng_swap]:
            coin.approve(pool, 2**256-1)

print(coin_a._storage.allowances.get())
print(coin_b._storage.allowances.get())

Add liquidity:

In [9]:
def _get_deposit_amounts(amount_per_token_usd, initial_prices, coins):

    precisions = [10 ** coin.decimals() for coin in coins]

    deposit_amounts = [
        amount_per_token_usd * precision * 10**18 // price
        for price, precision in zip(initial_prices, precisions)
    ]
    return deposit_amounts

deposit_amounts = _get_deposit_amounts(10**8, [10**18, initial_price], [coin_a, coin_b])
deposit_amounts

In [10]:
if ng_swap.totalSupply() == 0:
    lp_tokens_ng = ng_swap.add_liquidity(deposit_amounts, 0, sender=trader)
if old_swap_token.totalSupply() == 0:
    lp_tokens_old = old_swap.add_liquidity(deposit_amounts, 0, sender=trader)
    
lp_tokens_ng - lp_tokens_old

In [11]:
ng_swap

In [12]:
old_swap

## Compare trades

In [13]:
num_trades = 10000
data = {
    "timestamp": [],
    "i": [],
    "in_amt": [],
    "old_swap_out_amt": [],
    "ng_swap_out_amt": [],
    "old_swap_balances_a": [],
    "old_swap_balances_b": [],
    "ng_swap_balances_a": [],
    "ng_swap_balances_b": [],
    "old_swap_xcp_profit": [],
    "ng_swap_xcp_profit": [],
    "old_swap_D": [],
    "ng_swap_D": [],
    "old_swap_lp_price": [],
    "ng_swap_lp_price": [],
    "old_swap_virtual_price": [],
    "ng_swap_virtual_price": [],
    "old_swap_price_scale": [],
    "old_swap_price_oracle": [],
    "ng_swap_price_scale": [],
    "ng_swap_price_oracle": [],
    "ng_swap_xcp_oracle": [],
    "old_swap_last_price": [],
    "ng_swap_last_price": [],
    "ng_fee_receiver_balances_a": [],
    "ng_fee_receiver_balances_b": [],
    "old_fee_receiver_admin_balances": [],
    "old_swap_totalSupply": [],
    "ng_swap_totalSupply": [],
    "old_swap_fee": [],
    "ng_swap_fee": [],
}

In [14]:
if not os.path.exists(filename):

    coin_precisions = {0: 10**coin_a.decimals(), 1: 10**coin_b.decimals()}

    for _ in range(num_trades):
        
        i = random.randint(0, 1)
        trade_amount = random.randint(1, 10**4 * coin_precisions[i])
        
        out_old = old_swap.exchange(i, 1-i, trade_amount, 0, sender=trader)
        out_ng = ng_swap.exchange(i, 1-i, trade_amount, 0, sender=trader)
        
        # store data:
        data["timestamp"].append(boa.env.vm.state.timestamp)
        
        data["old_swap_out_amt"].append(out_old / coin_precisions[1-i])
        data["ng_swap_out_amt"].append(out_ng / coin_precisions[1-1])
        data["i"].append(i)
        data["in_amt"].append(trade_amount / coin_precisions[i])
        
        balances = old_swap._storage.balances.get()
        data["old_swap_balances_a"].append(balances[0] / coin_precisions[0])
        data["old_swap_balances_b"].append(balances[1] / coin_precisions[1])
        data["old_swap_D"].append(old_swap._storage.D.get() / 1e18)
        data["old_swap_xcp_profit"].append(old_swap._storage.xcp_profit.get() / 1e18)
        data["old_swap_price_oracle"].append(old_swap.price_oracle() / 1e18)
        data["old_swap_price_scale"].append(old_swap.price_scale() / 1e18)
        data["old_swap_last_price"].append(old_swap.last_prices() / 1e18)
        data["old_swap_virtual_price"].append(old_swap.virtual_price() / 1e18)
        data["old_swap_lp_price"].append(old_swap.lp_price() / 1e18)
        data["old_fee_receiver_admin_balances"].append(old_swap_token.balanceOf(fee_receiver))
        data["old_swap_totalSupply"].append(old_swap_token._storage.totalSupply.get())
        data["old_swap_fee"].append(old_swap.fee())
        
        balances = ng_swap._storage.balances.get()
        data["ng_swap_balances_a"].append(balances[0] / coin_precisions[0])
        data["ng_swap_balances_b"].append(balances[1] / coin_precisions[1])
        data["ng_swap_D"].append(ng_swap._storage.D.get() / 1e18)
        data["ng_swap_xcp_profit"].append(ng_swap._storage.xcp_profit.get() / 1e18)
        data["ng_swap_price_oracle"].append(ng_swap.price_oracle() / 1e18)
        data["ng_swap_xcp_oracle"].append(ng_swap.xcp_oracle() / 1e18)
        data["ng_swap_price_scale"].append(ng_swap.price_scale() / 1e18)
        data["ng_swap_last_price"].append(ng_swap.last_prices() / 1e18)
        data["ng_swap_virtual_price"].append(ng_swap.virtual_price() / 1e18)
        data["ng_swap_lp_price"].append(ng_swap.lp_price() / 1e18)
        data["ng_swap_totalSupply"] = ng_swap._storage.totalSupply.get()
        data["ng_swap_fee"].append(ng_swap.fee())

        with boa.env.anchor():
            ng_swap.remove_liquidity_one_coin(10**18, i, 0, sender=trader)
            data["ng_fee_receiver_balances_a"].append(coin_a.balanceOf(fee_receiver) / coin_precisions[0])
            data["ng_fee_receiver_balances_b"].append(coin_b.balanceOf(fee_receiver) / coin_precisions[1])
        
        boa.env.time_travel(random.randint(1, 1000))
        
    df = pd.DataFrame(data)
    df.set_index("timestamp", inplace=True)
    
else:
    
    df = pd.read_csv(filename, index_col="timestamp")

df
    

In [15]:
if not os.path.exists(filename):
    df.to_csv(filename)

Comparisons:

1. Differences in outputs
2. Differences Price oracle updates
3. Differences in price scale updates
4. Differences in abs(price_scale - price_oracle)
5. Differences in pool balances
6. Differences in D
7. Differences in xcp profit
8. Differences in admin balances

In [16]:
def print_diff(loc):
    
    calc_diff = lambda x: print(f"{x}: " + str(df.loc[loc, f"old_swap_{x}"] - df.loc[loc, f"ng_swap_{x}"]))
    i = df.loc[loc, "i"]
    print(f"Traded {i} > {1-i}, in amount: {df.loc[loc, 'in_amt']}")
    print("Difference between old_swap and ng_swap:")
    
    calc_diff("out_amt")
    calc_diff("balances_a")
    calc_diff("balances_b")
    calc_diff("D")
    calc_diff("xcp_profit")
    calc_diff("virtual_price")
    calc_diff("last_price")
    calc_diff("price_oracle")
    calc_diff("price_scale")
    calc_diff("lp_price")

In [17]:
fig, ax = plt.subplots(2, 2, figsize=(16, 9))
df[["old_swap_last_price", "ng_swap_last_price"]].plot(ax=ax[0][0], legend=False, ylabel="last_price")
df[["old_swap_price_oracle", "ng_swap_price_oracle"]].plot(ax=ax[0][1], legend=False, ylabel="price_oracle")
df[["old_swap_price_scale", "ng_swap_price_scale"]].plot(ax=ax[1][0], legend=False, ylabel="price_scale")
df[["old_swap_virtual_price", "ng_swap_virtual_price"]].plot(ax=ax[1][1], legend=False, ylabel="virtual_price")

fig.suptitle("blue=old, orange=ng")

It seems ng is more 'stable' ? let's compare scale deltas

In [18]:
ng_swap_scale_delta = df.ng_swap_price_oracle - df.ng_swap_price_scale
old_swap_scale_delta = df.old_swap_price_oracle - df.old_swap_price_scale

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ng_swap_scale_delta.plot(color='r', ax=ax[0])
old_swap_scale_delta.plot(color='b', ax=ax[1])


In [19]:
fig, ax = plt.subplots(1, 1,)
old_swap_scale_delta.plot.hist(ax=ax, bins=50, color='r')
ng_swap_scale_delta.plot.hist(ax=ax, bins=50, color='b', alpha=0.5)
ax.set_xlabel("Scale delta (price_scale - price_oracle)")

In [20]:
diff_output = (df.old_swap_out_amt).astype(float) - (df.ng_swap_out_amt).astype(float)
diff_output.plot()

The old implementation's price_scale changes more frequently whereas ng implementation's price scale does not do that as frequently. 

This is why we see such a difference in output swap amounts between the two implementations. This is not to say that the price scale in ng implementation does not change whatsoever: it does but not that often

In [31]:
fig, ax = plt.subplots(2, 2, figsize=(16, 9))
df[["old_swap_D", "ng_swap_D"]].plot(ax=ax[0][0], legend=False, ylabel="D")
df[["old_swap_xcp_profit", "ng_swap_xcp_profit"]].plot(ax=ax[0][1], legend=False, ylabel="xcp_profit")
df[["old_swap_lp_price", "ng_swap_lp_price"]].plot(ax=ax[1][0], legend=False, ylabel="lp_price")
df[["old_swap_fee", "ng_swap_fee"]].plot.hist(bins=50, alpha=0.5, ax=ax[1][1], legend=False, ylabel="fee")

fig.suptitle("blue=old, orange=ng")

In [22]:
ng_swap.lp_price()

In [23]:
sum(ng_swap._storage.balances.get()) / ng_swap.totalSupply()

How much admin fees are collected?

In [24]:
with boa.env.anchor():
    ng_swap.remove_liquidity_one_coin(10**18, i, 0, sender=trader)
    coin_a_bal = coin_a.balanceOf(fee_receiver) / coin_precisions[0]
    coin_b_bal = coin_b.balanceOf(fee_receiver) / coin_precisions[1]
    
print("ng_swap:", coin_a_bal, coin_b_bal)

In [25]:
with boa.env.anchor():
    old_swap.remove_liquidity(old_swap_token.balanceOf(fee_receiver), [0, 0], sender=fee_receiver)
    coin_a_bal = coin_a.balanceOf(fee_receiver) / coin_precisions[0]
    coin_b_bal = coin_b.balanceOf(fee_receiver) / coin_precisions[1]
    
print("old_swap:", coin_a_bal, coin_b_bal)

In [26]:
ng_swap.fee()

In [27]:
old_swap.fee()

Fee in ng pool is overall lower because of how state prices are calculated, which explains why ng_swap has marginally lower fee collected than the old implementation.